# Scaling Python with Azure ML and Dask

![Describe gif](media/describe.gif)

## Environment setup

This notebook assumes you are using an Azure ML Compute Instance with the default kernel `azureml_py36`. This contains many unneccesary packages. If you want to avoid a long image build time, you may want to create a new conda environment with the minimal packages needed for your scenario. 

It is important that the local environment matches the remote environment to avoid mismatch issues when submitting commands to the remote cluster. To help with this, we will use Azure ML Environments. 

In [1]:
# this is for a strange bug with compute instances 
import os

os.system('sudo cp /etc/nginx/nginx.conf setup/temp.conf') # stupid

nginx = ''

with open('setup/temp.conf') as f:
    for line in f.readlines():
        if 'websocket/|/ws/' in line:
            nginx += line.replace('websocket/|/ws/', 'websocket/|/ws')
        else:
            nginx += line
       
with open('setup/temp2.conf', 'w') as f:
    f.write(nginx)
    
os.system('sudo mv setup/temp2.conf /etc/nginx/nginx.conf')
os.system('sudo service nginx restart')
os.system('rm setup/temp.conf');

## Imports

Import all packages used in this notebook.

In [2]:
import os
import dask
import time
import joblib
import fsspec
import socket
import matplotlib

import pandas as pd
import dask.dataframe as dd
import matplotlib.pyplot as plt

from datetime import datetime
#from dask.distributed import Client
from IPython.core.display import HTML
#from dask_ml.xgboost import XGBRegressor

from azureml.widgets import RunDetails
from azureml.train.estimator import Estimator
from azureml.core.runconfig import MpiConfiguration
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core import Workspace, Experiment, Dataset, Environment, Model

%matplotlib inline

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception cannot import name '_DistributedTraining'.


In [3]:
t_start = time.time()

## Azure ML setup

Get the workspace.

In [4]:
ws = Workspace.from_config()
ws

Workspace.create(name='uks-azureml', subscription_id='6560575d-fa06-4e7d-95fb-f962e74efd7a', resource_group='copeters-rg')

### Enter your name

Enter your name and virtual network information.

In [6]:
### name
name        = 'cody'             # REPLACE

### vnet settings
vnet_rg     = ws.resource_group  # replace if needed
vnet_name   = 'uksouth-vnet'     # replace if needed
subnet_name = 'default'          # replace if needed

### azure ml names 
ct_name     = f'{name}-dask-ct'
exp_name    = f'{name}-dask-demo'

### trust but verify
verify = f'''
Name: {name}

vNET RG: {vnet_rg}
vNET name: {vnet_name}
vNET subnet name: {subnet_name}

Compute target: {ct_name}
Experiment name: {exp_name}
'''

print(verify)


Name: cody

vNET RG: copeters-rg
vNET name: uksouth-vnet
vNET subnet name: default

Compute target: cody-dask-ct
Experiment name: cody-dask-demo



### Create VM pool

Create Azure ML VM pool for creating remote dask cluster(s).

In [7]:
if ct_name not in ws.compute_targets:
    # create config for Azure ML cluster
    # change properties as needed
    config = AmlCompute.provisioning_configuration(
             vm_size                       = 'STANDARD_D14_V2', # 8 vCPUS 56 GB RAM 112 GB disk 
             max_nodes                     = 100,
             vnet_resourcegroup_name       = vnet_rg,              
             vnet_name                     = vnet_name,            
             subnet_name                   = subnet_name,          
             idle_seconds_before_scaledown = 300
    )
    ct = ComputeTarget.create(ws, ct_name, config)
    ct.wait_for_completion(show_output=True)    
else:
    ct = ws.compute_targets[ct_name]
    
ct

Creating
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned


AmlCompute(workspace=Workspace.create(name='uks-azureml', subscription_id='6560575d-fa06-4e7d-95fb-f962e74efd7a', resource_group='copeters-rg'), name=cody-dask-ct, id=/subscriptions/6560575d-fa06-4e7d-95fb-f962e74efd7a/resourceGroups/copeters-rg/providers/Microsoft.MachineLearningServices/workspaces/uks-azureml/computes/cody-dask-ct, type=AmlCompute, provisioning_state=Succeeded, location=uksouth, tags=None)

## Startup cluster

Start the run now. The first time, this will take 

In [10]:
script_params = {
    '--jupyter': True,
#    '--datastore': ws.datastores['codefiles']
}

In [11]:
# # of nodes 
nodes = 50

exp   = Experiment(ws, exp_name)
est   = Estimator('setup', 
                  compute_target          = ct, 
                  entry_script            = 'start.py',          # sets up Dask cluster
                  script_params           = script_params,
                  node_count              = nodes,        
                  distributed_training    = MpiConfiguration(),
                  custom_docker_image     = 'todrabas/aml_rapids:latest',
                  user_managed            = True
)

#run = next(exp.get_runs()) # use this to get existing run (if kernel restarted, etc)
run = exp.submit(est)
run

Experiment,Id,Type,Status,Details Page,Docs Page
cody-dask-demo,cody-dask-demo_1579672659_63813103,azureml.scriptrun,Starting,Link to Azure Machine Learning studio,Link to Documentation


In [12]:
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

### Connect to cluster

In [14]:
# port to forward the dask dashboard to on the compute instance
# we do not use 8787 because it is already in use 
dashboard_port = 4242
jupyter_port = 8888

print("waiting for scheduler node's ip")
while run.get_status() != 'Canceled' and 'scheduler' not in run.get_metrics():
    print('.', end ="")
    time.sleep(5)

if run.get_status() == 'Canceled':
    print('\nRun was canceled')
else:
    print(f'\nSetting up port forwarding...')
    os.system(f'killall socat') # kill all socat processes - cleans up previous port forward setups 
    os.system(f'setsid socat tcp-listen:{dashboard_port},reuseaddr,fork tcp:{run.get_metrics()["dashboard"]} &')
    os.system(f'setsid socat tcp-listen:{jupyter_port},reuseaddr,fork tcp:{run.get_metrics()["jupyter"]} &')

waiting for scheduler node's ip

Setting up port forwarding...
Cluster is ready to use.


<Client: 'tcp://10.3.0.11:8786' processes=60 threads=960, memory=7.10 TB>


In [ ]:
# build the dashboard link 
dashboard_url = f'https://{socket.gethostname()}-{dashboard_port}.{ws.get_details()["location"]}.instances.azureml.net/status'
HTML(f'<a href="{dashboard_url}">Dashboard link</a>')

In [ ]:
# build the dashboard link 
dashboard_url = f'https://{socket.gethostname()}-{dashboard_port}.{ws.get_details()["location"]}.instances.azureml.net/status'
HTML(f'<a href="{dashboard_url}">Dashboard link</a>')

## Cancel the run

In [40]:
c.close()
run.cancel()